<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import vari e inizializzazione pacchetti**

In [2]:
pip install textattack

### **NOTE IMPORTANTI**


> la funzione **Augmenter.augment(arg)** utilizza al suo interno **random**, peccato che non si puo settare il seed durante la fase di costruzione dell'oggetto o quando viene richiamata la funzione, questo ha comportato l'import di random per forzare il seed prima dell'esecuzione del codice.



In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit # pewr il testing

# **AMAZON**


In [83]:
dataset = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Amazon/Reviews.csv')
print(f"COLONNE:{dataset.columns}")
print(f"DIMENSIONE:{dataset.shape}")

COLONNE:Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
DIMENSIONE:(568454, 10)


Filtering del dataset dei dati duplicati e rimozione parti inutili di esso

In [84]:
rar=dataset.copy()
#print(rar.groupby(['Text']).count().shape)
rar.drop_duplicates(subset ="Text", 
                     keep = "last", inplace = True) 
rar.drop(['Id','ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator','HelpfulnessDenominator','Time'], axis=1, inplace=True)
print(f"COLONNE:{rar.columns}")
print(f"DIMENSIONE:{rar.shape}")

COLONNE:Index(['Score', 'Summary', 'Text'], dtype='object')
DIMENSIONE:(393579, 3)


In [85]:
from textattack.augmentation import WordNetAugmenter
from collections import OrderedDict
random.seed(33)

my_items = list()
number_of_trasformation_to_add=2

start_time = timeit.default_timer()

for row in rar.head(6).itertuples():
  #print(row.Text)
  augmenter = WordNetAugmenter(pct_words_to_swap=0.7, transformations_per_example=number_of_trasformation_to_add)
  result=augmenter.augment(row.Text)
  # print(f'original:{s} \n')
  #print(f'Augmented:{result[0]} \n')
  #Aggiungo al dataset finale la entry originale del dataset
  my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':row.Text})
  for i in range(number_of_trasformation_to_add):
    #Aggiungo al dataset finale una nuova entry augmentata 
    my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':result[i]})
  

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=rar.columns)
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=9.975929705000453
COLONNE:Index(['Score', 'Summary', 'Text'], dtype='object')
DIMENSIONE:(18, 3)


In [44]:
print(augmeted.head(6))

Empty DataFrame
Columns: [Score, Summary, Text]
Index: []


# **ATE ABISTA**

> WORD NET AUGMENTER da fare da circa zero unsado la classe WordSwapWordNet(lang=italian or it)



In [16]:
# import transformations, contraints, and the Augmenter
from textattack.transformations import WordSwapWordNet

from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

from textattack.augmentation import Augmenter

In [26]:
# Set up transformation using CompositeTransformation()
transformation = WordSwapWordNet(language='ita')
# Set up constraints
constraints = [RepeatModification(), StopwordModification()]# rimove la possibilità di sostituire parole gia sostituite,ed anche parole inutili dette "stopword"(the,as,a,in ecc)
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=1, transformations_per_example=2)
s = 'Ordinerò la pizza, ma il ristorante è al completo. Penso mangierò una pizza dal mio kebabbaro preferito!'
# Augment!
augmenter.augment(s)

['Ordinerò la mattone, ma il trattoria è al vestito. Penso mangierò una pittima dal mio kebabbaro coccolo!',
 'Ordinerò la palla, ma il trattoria è al grandissimo. Penso mangierò una bobina dal mio kebabbaro beniamino!']

Come si puo notare da questa frase italiana di esempio **WordSwapWordNet()** non è in grando di trovare nel suo db di parole italiane le varie coniugazioni dei vari verbi, il che limita la possiblità di variare il verbo a meno che esso non si all'infinito. **Questa è una grave limitazione del tool**, per analisi completa del db che crea i sinonimi visita:[link](http://compling.hss.ntu.edu.sg/omw/)

In [36]:
# Set up transformation using CompositeTransformation()
transformation = WordSwapWordNet(language='ita')
# Set up constraints
constraints = [RepeatModification(), StopwordModification()]# rimove la possibilità di sostituire parole gia sostituite,ed anche parole inutili dette "stopword"(the,as,a,in ecc)
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=1, transformations_per_example=2)
s = 'Vorrei un cane'
# Augment!
augmenter.augment(s)

['Vorrei un incompetente', 'Vorrei un inetto']

### **TEST GENERICI DI CODICE**

In [37]:
from nltk.corpus import wordnet
import textattack

l='eng'
word='dog'
synonyms = set()
for syn in wordnet.synsets(word, lang=l):
    for syn_word in syn.lemma_names(lang=l):
        if (
            (syn_word != word)
            and ("_" not in syn_word)
            and (textattack.shared.utils.is_one_word(syn_word))
        ):
            # WordNet can suggest phrases that are joined by '_' but we ignore phrases.
            synonyms.add(syn_word)
result=list(synonyms)

print(result)

['firedog', 'frump', 'trail', 'hound', 'blackguard', 'frank', 'cad', 'tail', 'track', 'andiron', 'hotdog', 'dog-iron', 'pawl', 'click', 'chase', 'detent', 'tag', 'wienerwurst', 'frankfurter', 'bounder', 'heel', 'wiener', 'weenie']
